In [1]:
import numpy as np


class Group:
    def __init__(self, members, candidate_items, ratings):
        # member ids
        self.members = sorted(members)
        
        # List of items that can be recommended.
        # These should not have been watched by any member of group.
        self.candidate_items = candidate_items

        self.actual_recos = []
        self.false_positive = []
        
        self.ratings_per_member = [np.size(ratings[member].nonzero()) for member in self.members]
        
        # AF
        self.grp_factors_af = []
        self.bias_af = 0
        self.precision_af = 0
        self.recall_af = 0
        self.reco_list_af = [] 
        
        # BF
        self.grp_factors_bf = []
        self.bias_bf = 0
        self.precision_bf = 0
        self.recall_bf = 0
        self.reco_list_bf = []
        
        # WBF
        self.grp_factors_wbf = []
        self.bias_wbf = 0
        self.precision_wbf = 0
        self.recall_wbf = 0
        self.weight_matrix_wbf = []
        self.reco_list_wbf = []
    
    # Verifies that given list of members can form a group w.r.t given data
    # ensure that there is at least 1 movie in training data that hasn't been
    # watched by any member of the group. Only these can be recommended.
    # call this method before calling group constructor.
    @staticmethod
    def find_candidate_items(ratings, members):
        if len(members) == 0:
            return []
        
        unwatched_items = np.argwhere(ratings[members[0]] == 0)
        for member in members:
            cur_unwatched = np.argwhere(ratings[member] == 0)
            unwatched_items = np.intersect1d(unwatched_items, cur_unwatched)
        
        return unwatched_items
    
    # generate groups of given size
    @staticmethod
    def generate_groups(ratings, test_ratings, num_users, count, size, disjoint=True):
        avbl_users = [i for i in range(num_users)]
        groups = []
        testable_threshold = 50
        
        iter_idx = 0
        while iter_idx in range(count):
            group_members = np.random.choice(avbl_users, size=size, replace=False)
            candidate_items = Group.find_candidate_items(ratings, group_members)
            non_eval_items = Group.non_testable_items(group_members, test_ratings)
            testable_items = np.setdiff1d(candidate_items, non_eval_items)
            
            if len(candidate_items) != 0 and len(testable_items) >= testable_threshold:
                groups += [Group(group_members, candidate_items, ratings)]

                if disjoint:
                    avbl_users = np.setdiff1d(avbl_users, group_members)
                iter_idx += 1
                
        return groups
    
    @staticmethod
    def non_testable_items(members, ratings): 
        non_eval_items = np.argwhere(ratings[members[0]] == 0)
        for member in members:
            cur_non_eval_items = np.argwhere(ratings[member] == 0)
            non_eval_items = np.intersect1d(non_eval_items, cur_non_eval_items)
        return non_eval_items
    
    def generate_actual_recommendations(self, ratings, threshold, is_debug=False):
        non_eval_items = Group.non_testable_items(self.members, ratings)
            
        items = np.argwhere(np.logical_or(ratings[self.members[0]] >= threshold, ratings[self.members[0]] == 0)).flatten()
        fp = np.argwhere(np.logical_and(ratings[self.members[0]] > 0, ratings[self.members[0]] < threshold)).flatten()
        for member in self.members:
            cur_items = np.argwhere(np.logical_or(ratings[member] >= threshold, ratings[member] == 0)).flatten()
            fp = np.union1d(fp, np.argwhere(np.logical_and(ratings[member] > 0, ratings[member] < threshold)).flatten())
            items = np.intersect1d(items, cur_items)
        
        items = np.setdiff1d(items, non_eval_items)

        self.actual_recos = items
        self.false_positive = fp
        if is_debug:
            print '\nT: ', self.actual_recos.size, ' F: ', self.false_positive.size

    def evaluate_af(self, is_debug=False):
        tp = float(np.intersect1d(self.actual_recos, self.reco_list_af).size)
        fp = float(np.intersect1d(self.false_positive, self.reco_list_af).size)
        
        try:
            self.precision_af = tp / (tp + fp)
        except ZeroDivisionError:
            self.precision_af = np.NaN

        try:
            self.recall_af = tp / self.actual_recos.size
        except ZeroDivisionError:
            self.recall_af = np.NaN

        if is_debug:
            print 'tp: ', tp
            print 'fp: ', fp
            print 'precision_af: ', self.precision_af
            print 'recall_af: ', self.recall_af

        return self.precision_af, self.recall_af, tp, fp

    def evaluate_bf(self, is_debug=False):
        tp = float(np.intersect1d(self.actual_recos, self.reco_list_bf).size)
        fp = float(np.intersect1d(self.false_positive, self.reco_list_bf).size)

        try:
            self.precision_bf = tp / (tp + fp)
        except ZeroDivisionError:
            self.precision_bf = np.NaN

        try:
            self.recall_bf = tp / self.actual_recos.size
        except ZeroDivisionError:
            self.recall_bf = np.NaN

        if is_debug:
            print 'tp: ', tp
            print 'fp: ', fp
            print 'precision_bf: ', self.precision_bf
            print 'recall_bf: ', self.recall_bf

        return self.precision_bf, self.recall_bf, tp, fp

    def evaluate_wbf(self, is_debug=False):
        tp = float(np.intersect1d(self.actual_recos, self.reco_list_wbf).size)
        fp = float(np.intersect1d(self.false_positive, self.reco_list_wbf).size)

        try:
            self.precision_wbf = tp / (tp + fp)
        except ZeroDivisionError:
            self.precision_wbf = np.NaN

        try:
            self.recall_wbf = tp / self.actual_recos.size
        except ZeroDivisionError:
            self.recall_wbf = np.NaN

        if is_debug:
            print 'tp: ', tp
            print 'fp: ', fp
            print 'precision_bf: ', self.precision_wbf
            print 'recall_bf: ', self.recall_wbf

        return self.precision_wbf, self.recall_wbf, tp, fp

In [2]:
import warnings
import numpy as np


class Aggregators:
    def __init__(self):
        pass

    @staticmethod
    def average(arr):
        return np.average(arr, axis = 0, weights = None)

    @staticmethod
    def average_bf(arr):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            arr[arr == 0] = np.nan
            return np.nanmean(arr, axis=0)
    
    @staticmethod
    def weighted_average(arr, weights):
        return np.average(arr, axis = 0, weights = weights)

In [3]:
from Aggregators import Aggregators
from Group import Group
from Config import Config
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as ps


# overflow warnings should be raised as errors
np.seterr(over='raise')


class GroupRec:
    def __init__(self):
        self.cfg = Config(r"./config.conf")
        
        # training and testing matrices
        self.ratings = None
        self.test_ratings = None

        self.groups = []
        
        # read data into above matrices
        self.read_data()
        
        self.num_users = self.ratings.shape[0]
        self.num_items = self.ratings.shape[1]
        
        # predicted ratings matrix based on factors.
        self.predictions = np.zeros((self.num_users, self.num_items))
        
        # output after svd factorization
        # initialize all unknowns with random values from -1 to 1
        self.user_factors = np.random.uniform(-1, 1, (self.ratings.shape[0], self.cfg.num_factors))
        self.item_factors = np.random.uniform(-1, 1, (self.ratings.shape[1], self.cfg.num_factors))

        self.user_biases = np.zeros(self.num_users)
        self.item_biases = np.zeros(self.num_items)
        
        # global mean of ratings a.k.a mu
        self.ratings_global_mean = 0

    # read training and testing data into matrices
    def read_data(self):
        column_headers = ['user_id', 'item_id', 'rating', 'timestamp']

        print 'Reading training data from ', self.cfg.training_file, '...'
        training_data = ps.read_csv(self.cfg.training_file, sep='\t', names=column_headers)

        print 'Reading testing data from ', self.cfg.testing_file, '...'
        testing_data = ps.read_csv(self.cfg.testing_file, sep='\t', names=column_headers)

        num_users = max(training_data.user_id.unique())
        num_items = max(training_data.item_id.unique())

        self.ratings = np.zeros((num_users, num_items))
        self.test_ratings = np.zeros((num_users, num_items))

        for row in training_data.itertuples(index=False):
            self.ratings[row.user_id - 1, row.item_id - 1] = row.rating

        for row in testing_data.itertuples(index=False):
            self.test_ratings[row.user_id - 1, row.item_id - 1] = row.rating

    # add list of groups
    def add_groups(self, groups):
        self.groups = groups
    
    # remove groups
    def remove_groups(self, groups):
        self.groups = []
    
    def predict_user_rating(self, user, item):
        prediction = self.ratings_global_mean + self.user_biases[user] + self.item_biases[item]
        prediction += self.user_factors[user, :].dot(self.item_factors[item, :].T)
        return prediction
    
    def predict_group_rating(self, grp, item, method):
        bias_grp = 0
        factors = np.nan
        if method == 'af':
            factors = grp.grp_factors_af; bias_grp = grp.bias_af
        elif method == 'bf':
            factors = grp.grp_factors_bf; bias_grp = grp.bias_bf
        elif method == 'wbf':
            factors = grp.grp_factors_wbf; bias_grp = grp.bias_wbf
        
        return self.ratings_global_mean + bias_grp + self.item_biases[item] \
                                        + np.dot(factors.T, self.item_factors[item])

    def predict_all_ratings(self):
        for user in range(self.num_users):
            for item in range(self.num_items):
                self.predictions[user, item] = self.predict_user_rating(user, item)
        
    # matrix factorization code
    def sgd_factorize(self):
        regularization = self.cfg.lambda_mf
        learning_rate = self.cfg.learning_rate_mf

        self.ratings_global_mean = np.mean(self.ratings[np.where(self.ratings != 0)])

        # solve for these for matrix ratings
        ratings_row, ratings_col = self.ratings.nonzero()
        num_ratings = len(ratings_row)
        
        print 'Doing matrix factorization...'
        try:
            for iter in range(self.cfg.max_iterations_mf):
                print 'Iteration: ', iter
                rating_indices = np.arange(num_ratings)
                np.random.shuffle(rating_indices)
                
                for idx in rating_indices:
                    user = ratings_row[idx]
                    item = ratings_col[idx]

                    pred = self.predict_user_rating(user, item)
                    error = self.ratings[user][item] - pred
                    
                    self.user_factors[user] += learning_rate * ((error * self.item_factors[item])
                                                    - (regularization * self.user_factors[user]))
                    self.item_factors[item] += learning_rate * ((error * self.user_factors[user])
                                                    - (regularization * self.item_factors[item]))
                    
                    self.user_biases[user] += learning_rate * (error - regularization * self.user_biases[user])
                    self.item_biases[item] += learning_rate * (error - regularization * self.item_biases[item])
            
                if self.cfg.is_debug:
                    self.sgd_mse()
            
        except FloatingPointError:
            print 'Floating point Error: '

    def sgd_mse(self):
        self.predict_all_ratings()
        predicted_training_ratings = self.predictions[self.ratings.nonzero()].flatten()
        actual_training_ratings = self.ratings[self.ratings.nonzero()].flatten()
        
        predicted_test_ratings = self.predictions[self.test_ratings.nonzero()].flatten()
        actual_test_ratings = self.test_ratings[self.test_ratings.nonzero()].flatten()
    
        training_mse = mean_squared_error(predicted_training_ratings, actual_training_ratings)
        print 'training mse: ', training_mse

        test_mse = mean_squared_error(predicted_test_ratings, actual_test_ratings)
        print 'test mse: ', test_mse

    # AF Method
    def af_runner(self, grps=None, aggregator=Aggregators.average):
        if grps is None:
            grps = self.groups
        
        # calculate factors
        for grp in grps:
            member_factors = self.user_factors[grp.members, :]
            member_biases = self.user_biases[grp.members]
        
            # aggregate the factors
            if aggregator == Aggregators.average:
                grp.grp_factors_af = aggregator(member_factors)
                grp.bias_af = aggregator(member_biases)
            elif aggregator == Aggregators.weighted_average:
                grp.grp_factors_af = aggregator(member_factors, weights=grp.ratings_per_member)
                grp.bias_af = aggregator(member_biases, weights=grp.ratings_per_member)

            self.make_recommendations_for_af(grp)

    def make_recommendations_for_af(self, grp):
        # predict ratings for all candidate items
        group_candidate_ratings = {}
        for idx, item in enumerate(grp.candidate_items):
            cur_rating = self.predict_group_rating(grp, item, 'af')
            if cur_rating > self.cfg.rating_threshold_af:
                group_candidate_ratings[item] = cur_rating

        # sort and filter to keep top 'num_recos_af' recommendations
        group_candidate_ratings = sorted(group_candidate_ratings.items(), key=lambda x: x[1], reverse=True)[:self.cfg.num_recos_af]
        grp.reco_list_af = np.array([rating_tuple[0] for rating_tuple in group_candidate_ratings])

    def make_recommendations_for_bf(self, grp):
        # predict ratings for all candidate items
        group_candidate_ratings = {}
        for idx, item in enumerate(grp.candidate_items):
            cur_rating = self.predict_group_rating(grp, item, 'bf')
            if cur_rating > self.cfg.rating_threshold_bf:
                group_candidate_ratings[item] = cur_rating

        # sort and filter to keep top 'num_recos_bf' recommendations
        group_candidate_ratings = sorted(group_candidate_ratings.items(), key=lambda x: x[1], reverse=True)[:self.cfg.num_recos_bf]
        grp.reco_list_bf = np.array([rating_tuple[0] for rating_tuple in group_candidate_ratings])

    def make_recommendations_for_wbf(self, grp):
        # predict ratings for all candidate items
        group_candidate_ratings = {}
        for idx, item in enumerate(grp.candidate_items):
            cur_rating = self.predict_group_rating(grp, item, 'wbf')
            if cur_rating > self.cfg.rating_threshold_wbf:
                group_candidate_ratings[item] = cur_rating

        # sort and filter to keep top 'num_recos_wbf' recommendations
        group_candidate_ratings = sorted(group_candidate_ratings.items(), key=lambda x: x[1], reverse=True)[:self.cfg.num_recos_wbf]
        grp.reco_list_wbf = np.array([rating_tuple[0] for rating_tuple in group_candidate_ratings])

    def get_weight_matrix(self, grp, watched_items):
        wt = []
        for item in watched_items:
            rated = np.argwhere(self.ratings[:, item] != 0)  # list of users who have rated this movie
            watched = np.intersect1d(rated, grp)  # list of group members who have watched this movie
            std_dev = np.std(filter(lambda a: a != 0, self.ratings[:, item]))  # std deviation for the rating of the item
            wt += [len(watched) / float(len(grp.members)) * 1 / (1 + std_dev)]  # list containing diagonal elements
        W = np.diag(wt)  # diagonal weight matrix
        return W

    # BF/WBF Method
    def bf_runner(self, grps=None, aggregator=Aggregators.average_bf, is_wbf=False):
        # aggregate user ratings into virtual group
        # calculate factors of group
        lamb = self.cfg.lambda_mf
        for grp in grps:
            all_movies = np.arange(len(self.ratings.T))
            watched_items = sorted(list(set(all_movies) - set(grp.candidate_items)))

            group_rating = self.ratings[grp.members, :]
            agg_rating = aggregator(group_rating)
            s_g = []
            for j in watched_items:
                s_g.append(agg_rating[j] - self.ratings_global_mean - self.item_biases[j])

            # creating matrix A : contains rows of [item_factors of items in watched_list + '1' vector]
            A = np.zeros((0, self.cfg.num_factors))  # 3 is the number of features here = K

            for item in watched_items:
                A = np.vstack([A, self.item_factors[item]])
            v = np.ones((len(watched_items), 1))
            A = np.c_[A, v]

            if is_wbf:
                W = self.get_weight_matrix(grp, watched_items)
                factor_n_bias = np.dot(np.linalg.inv(np.dot(np.dot(A.T, W),A) + lamb * np.identity(self.cfg.num_factors + 1)), np.dot(np.dot(A.T, W), s_g))
                grp.grp_factors_wbf = factor_n_bias[:-1]
                grp.bias_wbf = factor_n_bias[-1]
                self.make_recommendations_for_wbf(grp)
            else:
                factor_n_bias = np.dot(np.linalg.inv(np.dot(A.T, A) + lamb * np.identity(self.cfg.num_factors + 1)), np.dot(A.T, s_g))
                grp.grp_factors_bf = factor_n_bias[:-1]
                grp.bias_bf = factor_n_bias[-1]
                self.make_recommendations_for_bf(grp)

    def evaluation(self):
        # For AF
        af_precision_list = []
        af_recall_list = []
        print "\n#########-------For AF-------#########"
        for grp in self.groups:
            grp.generate_actual_recommendations(self.test_ratings, self.cfg.rating_threshold_af, self.cfg.is_debug)
            (precision, recall, tp, fp) = grp.evaluate_af(self.cfg.is_debug)
            af_precision_list.append(precision)
            af_recall_list.append(recall)
        
        af_mean_precision = np.nanmean(np.array(af_precision_list))
        af_mean_recall = np.nanmean(np.array(af_recall_list))
        print '\nAF method: mean precision: ', af_mean_precision
        print 'AF method: mean recall: ', af_mean_recall

        # For BF
        bf_precision_list = []
        bf_recall_list = []
        print "\n#########-------For BF-------#########"
        for grp in self.groups:
            grp.generate_actual_recommendations(self.test_ratings, self.cfg.rating_threshold_bf, self.cfg.is_debug)
            (precision, recall, tp, fp) = grp.evaluate_bf(self.cfg.is_debug)
            bf_precision_list.append(precision)
            bf_recall_list.append(recall)

        bf_mean_precision = np.nanmean(np.array(bf_precision_list))
        bf_mean_recall = np.nanmean(np.array(bf_recall_list))
        print '\nBF method: mean precision: ', bf_mean_precision
        print 'BF method: mean recall: ', bf_mean_recall

        # For WBF
        wbf_precision_list = []
        wbf_recall_list = []
        print "\n#########-------For WBF-------#########"
        for grp in self.groups:
            grp.generate_actual_recommendations(self.test_ratings, self.cfg.rating_threshold_wbf, self.cfg.is_debug)
            (precision, recall, tp, fp) = grp.evaluate_wbf(self.cfg.is_debug)
            wbf_precision_list.append(precision)
            wbf_recall_list.append(recall)

        wbf_mean_precision = np.nanmean(np.array(wbf_precision_list))
        wbf_mean_recall = np.nanmean(np.array(wbf_recall_list))
        print '\nWBF method: mean precision: ', wbf_mean_precision
        print 'WBF method: mean recall: ', wbf_mean_recall

In [4]:
def run_all_methods(self, grps):
    if grps is None:
        grps = self.groups

    self.af_runner(grps, Aggregators.weighted_average)
    self.bf_runner(grps, Aggregators.average_bf)
    self.bf_runner(grps, Aggregators.average_bf, is_wbf=True)  # For WBF

    # evaluation
    self.evaluation()

GroupRec.run_all_methods = run_all_methods

In [5]:
if __name__ == "__main__":
    gr = GroupRec()

    # factorize matrix
    gr.sgd_factorize()
    
    # add groups or generate random groups of given size
    groups = []

    # members = [475, 549, 775]
    # candidate_items = Group.find_candidate_items(gr.ratings, members)
    # if len(candidate_items) != 0:
    #   groups = [Group(gr.cfg, members, candidate_items, gr.ratings)]

    # disjoint means none of the groups shares any common members
    small_groups = Group.generate_groups(gr.ratings, gr.test_ratings, gr.num_users, gr.cfg.no_of_small_grps, gr.cfg.small_grp_size, disjoint=False)
    medium_groups = Group.generate_groups(gr.ratings, gr.test_ratings, gr.num_users, gr.cfg.no_of_medium_grps, gr.cfg.medium_grp_size, disjoint=False)
    large_groups = Group.generate_groups(gr.ratings, gr.test_ratings, gr.num_users, gr.cfg.no_of_large_grps, gr.cfg.large_grp_size, disjoint=False)
    
    group_set = [small_groups, medium_groups, large_groups]
    group_type = ['small', 'medium', 'large']
    
    for idx, groups in enumerate(group_set):
        if groups is []:
            continue
        
        # generated groups
        n = len(groups) if gr.cfg.is_debug else 5
        print '\n******* Running for ', group_type[idx], ' groups *************'
        print 'generated groups (only %d are getting printed here): ' % n
        for group in groups[:n]:
            print(group.members)
        
        gr.add_groups(groups)
        gr.run_all_methods(groups)
        gr.remove_groups(groups)

Reading training data from  ./data/u1.base ...
Reading testing data from  ./data/u1.test ...
Doing matrix factorization...
Iteration:  0

******* Running for  small  groups *************
generated groups (only 5 are getting printed here): 
[82, 93, 371]
[206, 288, 529]
[57, 145, 267]
[63, 345, 670]
[322, 325, 543]

#########-------For AF-------#########

AF method: mean precision:  0.828251881561
AF method: mean recall:  0.0833815853777

#########-------For BF-------#########

BF method: mean precision:  0.740671332636
BF method: mean recall:  0.0409910891095

#########-------For WBF-------#########

WBF method: mean precision:  0.798686223968
WBF method: mean recall:  0.141507108783

******* Running for  medium  groups *************
generated groups (only 5 are getting printed here): 
[107, 303, 306, 679, 708]
[81, 176, 350, 468, 631]
[68, 258, 263, 726, 766]
[84, 413, 527, 818, 880]
[145, 196, 294, 385, 622]

#########-------For AF-------#########

AF method: mean precision:  0.80432